# Output File Analysis - Multiple SRRs - Donor to Recipient Workflows

## Variables

In [10]:
## Variables
srr_names <- c("SRR5090597", "SRR5090599")
donor_name <- "hpv16"
recipient_name <- "UCSChg38"
inputs_folder <- "../../wallaby/workflows/cromwell-final-outputs/"
donor_ref_genome <- "../../wallaby/data/ref_genomes/hpv/HPV16.fa"
recipient_ref_genome <- "../../wallaby/data/ref_genomes/human/hg38.fa"

In [10]:
## Variables
srr_names <- c("SRR1609136", "SRR1609137", "SRR1609138", "SRR1609139", "SRR1609140", "SRR1609141", "SRR1609142", "SRR1609143", "SRR1609144", "SRR1609145", "SRR1609146", "SRR1609147", "SRR1609148", "SRR1609149", "SRR1609150", "SRR1609151")
donor_name <- "hpv16"
recipient_name <- "UCSChg38"
inputs_folder <- "../../wallaby/workflows/cromwell-final-outputs/"
donor_ref_genome <- "../../wallaby/data/ref_genomes/hpv/HPV16.fa"
recipient_ref_genome <- "../../wallaby/data/ref_genomes/human/hg38.fa"

## Import Libraries

In [11]:
## Setup Environment
# Enable multithreading when possible (library dependent)
options(Ncpus = parallel::detectCores())
Sys.setenv(OMP_NUM_THREADS=toString(parallel::detectCores()))
Sys.setenv(OMP_THREAD_LIMIT=toString(parallel::detectCores()))
Sys.setenv(OMP_NUM_THREADS=parallel::detectCores())
Sys.setenv(OMP_THREAD_LIMIT=parallel::detectCores())

## Load or install and load all libraries
suppressPackageStartupMessages(library("pacman", character.only = TRUE))

# List of CRAN packages to either Load, or Install and Load
pacman::p_load(dplyr, ggplot2, shiny, shinyLP, DT,  ggrepel,  tidyr, 
               data.table, kableExtra, knitr, IRdisplay, stringr, install = FALSE)

# List of Bioconductor packages to either Load, or Install and Load
pacman::p_load(BSgenome, BSgenome.Hsapiens.UCSC.hg38, GenomicFeatures, 
               GenomicAlignments,  Rsubread,  Rsamtools, bamsignals,  
               rtracklayer, GenomicRanges, org.Hs.eg.db, Organism.dplyr,
               TxDb.Hsapiens.UCSC.hg38.knownGene, regioneR, karyoploteR,
               seqinr, Repitools, Gviz, Biostrings, install = FALSE)

## Load Files for Analysis

In [12]:
# Select crossings to import and order of visualization
crossings <- c("UMd_MMr", "MUd_MMr", "UMd_MUr", "MUd_UMr", "MMd_UMr", "MMd_MUr")

In [13]:
# Load all .bed files for all srrs created by the cromwell workflow

# Define variables to hold all srrs
recip_granges_all_srrs <- list()

# Loop over each srr
for (srr in srr_names) {
    
    # For each single srr
    recip_granges <- list()
    recip_granges_crossings <- list()

    # Load all .bed files created by the cromwell workflow
    for (cross in crossings){
        # list all recipient files
        recip_file <- list.files(inputs_folder,
                                  pattern=paste(srr, 
                                                '-to-',
                                                recipient_name,
                                                "_", cross, 
                                                ".bed", 
                                                sep = ""), 
                                  recursive = TRUE, 
                                  full.names = TRUE)

        # check whether the file exists and add if it does
        if (!identical(recip_file, character(0))) {
            recip_granges[[cross]] <- import(recip_file)
            recip_granges_crossings[[cross]] <- cross
        }
        
        # add recip_granges for this srr to the _all object
        recip_granges_all_srrs[[srr]] <- recip_granges
    }
}

## Helper Functions

In [14]:
# Create a sqlite database from TxDb and corresponding Org packages
# The database provides a convenient way to map between gene, transcript, and protein identifiers.
src <- suppressMessages(suppressWarnings(src_organism("TxDb.Hsapiens.UCSC.hg38.knownGene")))
# Create a full genome sequences for Homo sapiens (Human) as provided by UCSC (hg38, based on GRCh38.p12)
Hsapiens <- BSgenome.Hsapiens.UCSC.hg38

In [15]:
# Function to Create a Table mapping ranges of overlapping paired-end crossings 
summary_table_recipient <- function(granges, 
                                    granges_labels, 
                                    min_num_crossings = 3,
                                    min_num_reads = 3, 
                                    src,
                                    Hsapiens) {
    
    # convert all granges to dataframes
    granges_df <- lapply(granges, annoGR2DF)
    # assign all granges labels (crossing names) as each dataframe's name
    names(granges_df) <- granges_labels
    # flatten all dataframes into a single dataframe, 
    # and use their's respective name as an identifier in a new column named 'crossing'
    merged_df <- bind_rows(granges_df, .id = "crossing")
    # convert the data.frame to a data.table
    merged_dt <- as.data.table(merged_df)

    # assign each row to a group, based on whether their ranges overlap
    merged_dt[,group := { ir <- IRanges(start, end); subjectHits(findOverlaps(ir, reduce(ir))) }, by = chr]
    # aggregate results by group, and add additional aggregated columns
    merged_final <- merged_dt[, list(start = min(start), 
                                     stop = max(end), 
                                     num_crossings = length(unique(list(crossing)[[1]])),
                                     unique_crossings = list(unique(crossing)),
                                     num_reads = length(list(name)[[1]])
                                     ), by = list(group,chr)]
    
    # filter results using a minimum number of reads and/or crossings
    merged_final <- merged_final[merged_final[, num_reads > (min_num_reads - 1)]]
    merged_final <- merged_final[merged_final[, num_crossings > (min_num_crossings - 1)]]
    
    # there might be no matches, so check for that before looking for a gene name
    if (nrow(merged_final) != 0) {
        # use the src database to look for the gene names or each crossing overlap region
        # then, add it as a new column
        merged_final$gene_name <- apply(merged_final, 1, FUN = function(x) toString(
            unique(unlist(suppressWarnings(annoGR2DF(
                                    transcripts(src, 
                                                 filter=~(GRangesFilter(
                                                     GenomicRanges::GRanges(
                                                         paste(toString(x["chr"]), ":", 
                                                               as.integer(x["start"]), "-", 
                                                               as.integer(x["stop"]), sep = "")))), 
                                                 columns=c("symbol")))$symbol)))))
                                        
        # use the Hsapiens databse to look up the DNA sequence for each crossing overlap region
        # then, add it as a new column
        merged_final$sequence <- apply(merged_final, 1, FUN = function(x) toString(getSeq(Hsapiens, 
                                                                           toString(x["chr"]), 
                                                                           start = as.integer(x["start"]), 
                                                                           end = as.integer(x["stop"]))))
        # delete the 'group' column
        merged_final <- merged_final[, !"group"]
        # add an ID to each row
        merged_final <- merged_final[, id := .I]
        setcolorder(merged_final, c("id", "chr", "start", "stop", 
                                    "num_crossings", "unique_crossings", 
                                    "num_reads", "gene_name", "sequence"))
                                       
    # if there are no matches, write an NA row
    } else {
            return(data.table(id = "<NA>",
                              chr = "<NA>", 
                              start = 0, 
                              stop = 0, 
                              num_crossings = 0, 
                              unique_crossings = "<NA>", 
                              num_reads = 0, 
                              gene_name = "<NA>", 
                              sequence = "<NA>")
                  )
        }

    return(merged_final)
}

In [16]:
# Function to Create a Summary Table for many SRRs
srrs_summary_table_recipient <- function(granges_list, 
                                         min_num_crossings = 3,
                                         min_num_reads = 3, 
                                         src,
                                         Hsapiens) {

    # store all tables in a single list
    srrs_list <- list()
    
    # iterate over all granges
    for (srr in names(granges_list)) {

        # create a summary table for each granges object
        granges <- unname(granges_list[srr][[1]])
        granges_labels <- str_split(names(granges_list[srr][[1]]), " ")
        summary_table <- summary_table_recipient (granges = granges,
                                                  granges_labels = granges_labels,
                                                  min_num_crossings = 1, 
                                                  min_num_reads = 3, 
                                                  src = src, 
                                                  Hsapiens = Hsapiens)
        
        # add a column for the srr
        summary_table$srr <- srr
        # add the table to the list
        srrs_list[[srr]] <- summary_table
    }
    
    # bind all the tables by row
    srrs_summary_table <- do.call(rbind, c(srrs_list, fill=TRUE))
    # reorder the table
    setcolorder(srrs_summary_table, c("srr", "id", "chr", "start", "stop", 
                                      "num_crossings", "unique_crossings", 
                                      "num_reads", "gene_name", "sequence"))

    # stylize the output
    srrs_summary_table %>%
        kable("html") %>%
        kable_styling(bootstrap_options = "striped", full_width = F, position = "left") %>%
        column_spec(10, width = "30em", width_max = "30em") %>%
        as.character() %>%
        display_html()     
}

## Recipient Analysis

In [17]:
# Aggregated view of all overlapping crossings for the potential recipient for all srrs
srrs_summary_table <- srrs_summary_table_recipient (granges_list = recip_granges_all_srrs, 
                                                    min_num_crossings = 1,
                                                    min_num_reads = 2, 
                                                    src = src,
                                                    Hsapiens = Hsapiens)

srr,id,chr,start,stop,num_crossings,unique_crossings,num_reads,gene_name,sequence
SRR5090597,1,chr1,150577086,150577493,6,"UMd_MMr, MUd_MMr, UMd_MUr, MUd_UMr, MMd_UMr, MMd_MUr",68,MCL1,CCCTACCATCTTCACTAAATCTAAAAGTCCTCCTCCATAGCTTCCCAAACAAAGTTTGTTTGTTGCTGAAACTGAACTTTGCTTCTTTCAGACAGTGACTCTTCAATCAATGGGGAGCACTCTTCCCATGTATTTATTCTTGTTAGCCATAATCCTCTTGCCACTTGCTTTTCTGGCTAGGTTGCTAGGGTGCAACTCTAGGAAGTTACAGCTTGGAGTCCAACTGCATAAACTGGTTTTGGTGGTGGTGGTGGTTGGTTAAAAGTCAACTATTGCACTTACAGTAAGGCTATCTTATTAGATATGCCAAACCAGCTCCTACTCCAGCAACACCTGCAAAAGCCAGCAGCACATTCCTGATGCCACCTTCTAGGTCCTCTACATGGAAGAACTCCACAAACCCATCCT
SRR5090597,2,chr1,150578243,150578495,5,"UMd_MMr, MUd_MMr, UMd_MUr, MUd_UMr, MMd_MUr",129,MCL1,CCCAGCCTCTTTGTTTAACTAGCCAGTCCCGTTTTGTCCTTACGAGAACGTCTGTGATACTTTCTGCTAATGGTTCGATGCAGCTTTCTTGGTTTATGGTCTTCAAGTGTTTAGCCACAAAGGCACCAAAAGAAATGAGAGTCACAATCCTGCCCCAGTTTGTTACGCCGTCGCTGAAAACATGGATCATCACTCGAGACAACGATTTCACATCGTCTTCGTTTTTGATGTCCAGTTTCCGAAGCATGCCTGA
SRR5090597,3,chr8,127828233,127828354,2,"MUd_MMr, MMd_MUr",3,PVT1,CTTCTTACTGAGTGTCTAGTTCCTTGTTGTACAGGTGAGGGAAACTGAGGCCCAGAGTGGATGTGACGTTTCCGAAGGAGAGGCTCTGTAAAGTTCTGAGTAGGTGAGACTTATTATTAGCT
SRR5090599,1,chr1,206940489,206940589,4,"UMd_MMr, MUd_MMr, UMd_MUr, MUd_UMr",5,PIGR,CTGGGAAGACCGCCAGCAGGCAGGTGAGCACGAAGAGCAGCATTGCTGGTGGGTCCCGAGCGCCGCACCACTCAGGCCGACTTCTCCTGTGCAATGCTGAA
SRR5090599,2,chr1,206949934,206950244,4,"UMd_MMr, MUd_MMr, UMd_MUr, MUd_UMr",13,,GCCACCCTCCAGGCTCTAGACGATAATCCGTTGCTTGCCTCTTCTAGCTTCTGGTTGCTGCTGACATTCTTTGGTGTTCTTTGGCACGTAGCTGTGTCACTCCAACCTCTGCCTTCATAGTCACATTGGCTTCTGTGTATGTCTCAAATAACTCTCTGTTTCTCTGTTATATGGATGGTATTTAGGGCCCACCCAGCTAATCCAAGATGATGTCCTCATCTCAAGATCCTGAACTTCATCACATCTACAAAGACTCTTTTCCAAATAAGGTCACACTTCTGGGCTCCAGAGATTGAGACATAGACAAACCT
SRR5090599,3,chr1,206939240,206939381,3,"MUd_MMr, UMd_MUr, MUd_UMr",4,PIGR,CTCCGGGAAGTTGGTGAGGTTAGCCCTGCCTGCATATTTGCTGGAGACGTAGCCCTCCGAGGAGATGAGGGTTATGCAGCCACCTCTAGCTCCCTGCCGGCACCAGTACTTCCGGGTGTGCCGGTTGACAGAGGTGGGTGGG
